In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rc("font", size=18)
import matplotlib.pyplot as plt
import yt
import numpy as np
from yt.units import kboltz, mp

In [ ]:
X_H = 0.76 # hydrogen mass fraction

Later we will make a cut on particles with kT > 30 eV, so figure out what the corresponding temperature in Kelvin is:

In [ ]:
kT_low = yt.YTQuantity(0.03, "keV") # 30 eV
T_low = float(kT_low.to_equivalent("K", "thermal"))

Load up the dataset:

In [ ]:
fn = "my_dataset.h5"
ds = yt.load(fn)

Here I'm assuming there's no electron number density field, so I use the `ElectronAbundance` field here to determine the electron number density:

In [ ]:
def _n_e(field, data):
    return data["PartType0", "Density"]*data["PartType0", "ElectronAbundance"]*X_H/mp
ds.add_field(("PartType0", "n_e"), function=_n_e, units="cm**-3", particle_type=True)

Now that I have an electron density field I can set up an entropy field:

In [ ]:
def _entropy(field, data):
    return (kboltz*data["PartType0", "Temperature"]).to("keV")*data["PartType0", "n_e"]**(-2./3.)
ds.add_field(("PartType0", "entropy"), function=_entropy, units="keV*cm**2", particle_type=True)

Create a sphere, you can change the center and radius parameters as needed:

In [ ]:
center = "max" # or something
radius = (1.0, "Mpc") # or something
sp = ds.sphere(center, radius)

and now we can make a cut region where we cut out the particles with kT < 30 eV:

In [ ]:
cr = sp.cut_region(["obj['PartType0', 'Temperature'] > %s" % T_low])

These are the fields I want to profile:

In [ ]:
fields = [("PartType0", "Density"), 
          ("PartType0", "Temperature"), 
          ("PartType0", "Metallicity"), 
          ("PartType0", "n_e"),
          ("PartType0", "Entropy")]

and I'm going to set the radial extrema for each particle type and the units of various things:

In [ ]:
dm_extrema = {("PartType1", "particle_radius"): (2.0, 1000.0)}
gas_extrema = {("PartType0", "particle_radius"): (2.0, 1000.0)}
star_extrema = {("PartType4", "particle_radius"): (2.0, 1000.0)}
dm_units = {("PartType1", "particle_radius"): "kpc", 
            ("PartType1", "particle_mass"): "Msun"}
gas_units = {("PartType0", "particle_radius"): "kpc"}
star_units = {("PartType4", "particle_radius"): "kpc", 
              ("PartType4", "particle_mass"): "Msun"}

I will make two profiles, one from the sphere itself and another from the part of the sphere that only has hot gas with kT > 30 eV:

In [ ]:
ps = sp.profile(("PartType0", "particle_radius"), fields, extrema=gas_extrema,
                logs={"particle_radius": True}, units=gas_units, n_bins=60, 
                weight_field=("PartType0", 'particle_ones'))
pc = cr.profile(("PartType0", "particle_radius"), fields, extrema=gas_extrema,
                logs={"particle_radius": True}, units=gas_units, n_bins=60, 
                weight_field=("PartType0", 'particle_ones'))

Electron density plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x, ps["n_e"], label='All Particles')
ax.plot(pc.x, pc["n_e"], label='Particles w/ kT > 30 eV')
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{n_e\ (cm^{-3})}$")
ax.legend()
fig.savefig("halo_density.png")

Temperature plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x.to("kpc"), (kboltz*ps["Temperature"]).to("keV"), label='All Particles')
ax.plot(pc.x.to("kpc"), (kboltz*pc["Temperature"]).to("keV"), label='Particles w/ kT > 30 eV')
ax.set_xscale("log")
ax.set_xlabel("r (kpc)")
ax.set_ylabel("T (keV)")
ax.legend()
fig.savefig("halo_kT.png")

Metallicity plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x.to("kpc"), ps["metallicity"], label='All Particles')
ax.plot(pc.x.to("kpc"), pc["metallicity"], label='Particles w/ kT > 30 eV')
ax.set_xscale('log')
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{Z\ (Z_\odot)}$")
ax.legend()
fig.savefig("halo_Z.png")

Entropy plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x.to("kpc"), ps["entropy"], label='All Particles')
ax.plot(pc.x.to("kpc"), pc["entropy"], label='Particles w/ kT > 30 eV')
ax.set_xscale('log')
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{S\ (keV\ cm^2)}$")
ax.legend()
fig.savefig("halo_entropy.png")

I now want to set the units of the gas particle mass to $M_\odot$:

In [ ]:
gas_units["PartType0", "particle_mass"] = "Msun"

And I will now make accumulated mass profiles of gas, dark matter, and stars:

In [ ]:
# DM
pmd = sp.profile(("PartType1", "particle_radius"), [("PartType1", "particle_mass")], 
                 extrema=dm_extrema, n_bins=128, weight_field=None, accumulation=True,
                 units=dm_units)
# Gas
pmg = sp.profile(("PartType0", "particle_radius"), [("PartType0", "particle_mass")], 
                 extrema=gas_extrema, n_bins=128, weight_field=None, accumulation=True,
                 units=gas_units)
# Stars
pms = sp.profile(("PartType4", "particle_radius"), [("PartType4", "particle_mass")], 
                 extrema=star_extrema, n_bins=128, weight_field=None, accumulation=True,
                 units=star_units)

And now I can make the mass plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(pmd.x, pmd["particle_mass"]+pmg["particle_mass"]+pms["particle_mass"], 
        label='Total Mass')
ax.plot(pmg.x, pmg["particle_mass"], label='Gas Mass')
ax.set_xscale('log')
ax.set_yscale("log")
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{M\ (M_\odot)}$")
ax.legend()
fig.savefig("halo_masses.png")